In [25]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [38]:
product_type = "ambient"
num_time_periods = 365

In [39]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

,ARTICLE_ID,TEMPERATURE_ZONE,CATEGORY_LEVEL_1,CATEGORY_LEVEL_2,VOLUME_M3_PER_CU,MEAN_SHELF_LIFE,CU_PER_TU,ORDERING_COST_FIXED,ORDERING_COST_PER_TU,SALES_MARGIN_PER_CU,CLEARING_COST_PER_CU,MINIMUM_ORDER_QUANTITY_TU,MAXIMUM_ORDER_QUANTITY_TU
1,2cafa187,ambient,Backen & Dessert,Back-Dekoration,0.000198,1000,4,6.0,0.5,0.08,0.3,0,NaN
3,2c3d0bc9,ambient,Obst,"Bananen, Kiwis & Exoten",0.002580,2,16,14.6,2.1,0.33,1.4,1,NaN
4,4b3fe1b3,ambient,Alkoholische Getränke,Spirituosen,0.001664,1000,19,16.5,2.5,0.39,1.6,0,NaN
6,2100c4a3,ambient,Baby & Kind,Gläschen & Snacks,0.000354,1000,15,13.7,2.0,0.31,1.3,0,NaN
7,5c65cb41,ambient,Haushalt,Waschen & Flecken,0.002499,1000,8,8.8,1.1,0.16,0.7,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,f73c3775,ambient,Süßes & Salziges,Nüsse,0.000602,1000,1,4.0,0.2,0.02,0.1,0,NaN
9396,83f25ff6,ambient,"Brot, Cerealien & Aufstriche",Müsli & Müsliriegel,0.001984,1000,15,14.1,2.0,0.32,1.3,0,NaN
9398,2339bcf1,ambient,Backen & Dessert,Mehl & Speisestärke,0.000629,1000,12,11.6,1.6,0.24,1.0,0,NaN
9400,694e718a,ambient,Tier,Katze,0.002285,1000,22,18.9,2.9,0.46,1.9,0,NaN


In [28]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 20
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 50

In [29]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [30]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [31]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [32]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [33]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

#### Heuristic

In [34]:
demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
demand = dict((date_to_index[key],value) for (key,value) in demand.items())

In [35]:
class CostMatrixCell:
    def __init__(self) -> None:
        self.orders = {}
        self.order_date = None
        self.period_end = None
        self.cost = 0
        self.avg_cost = 0
        self.extra_orders = {}
        self.clearance_cost = 0

class Order:
    def __init__(self) -> None:
        self.item = None
        self.quantity_tu = None
        self.date = None

In [36]:
n = len(demand.keys())
cost_matrix = [[CostMatrixCell() for __ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        if(i>j):
            cost_matrix[i][j] = None
        else:
            cost_matrix[i][j].order_date = i
            cost_matrix[i][j].period_end = j

def getOrders(cell):
    clearance_cost = 0
    for t in range(cell.order_date, cell.period_end + 1):
        for item in demand[t].keys():
            if(item not in cell.orders.keys()):
                cell.orders[item] = 0
            if(cell.order_date + shelf_life[item] < t):
                # item cannot be ordered on order_date because it would go bad by t
                if(t not in cell.extra_orders.keys()):
                    cell.extra_orders[t] = {}
                cell.extra_orders[t][item] = math.ceil(demand[t][item] / cu_per_tu[item])
                clearance_cost += ((math.ceil(demand[t][item] / cu_per_tu[item]) * cu_per_tu[item]) - demand[t][item]) * clearing_cost_per_cu[item]
            else:   
                cell.orders[item] += demand[t][item]
                # cell.orders[item] += math.ceil(demand[t][item] / cu_per_tu[item])
    for item in cell.orders.keys():
        cell.orders[item] = math.ceil(cell.orders[item]/cu_per_tu[item])
    cell.clearance_cost = clearance_cost

def fetchExtraVolume(cell, t):
    extra_volume = 0
    if(t in cell.extra_orders.keys()):
        for item in cell.extra_orders[t].keys():
            extra_volume += cell.extra_orders[t][item] * cu_per_tu[item] * volume_per_cu[item] 
    return extra_volume
            
def getCost(cell):
    getOrders(cell)
    cost = cell.clearance_cost
    volume = 0
    for item in cell.orders.keys():
        cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        for extra_order_date in cell.extra_orders.keys():
            for extra_item in cell.extra_orders[extra_order_date].keys():
                cost += (ordering_cost_per_tu[item] * cell.orders[item]) + ordering_cost_fixed[item]
        volume += cell.orders[item] * cu_per_tu[item] * volume_per_cu[item]
    for t in range(cell.order_date,cell.period_end+1):
        if(volume + fetchExtraVolume(cell, t) > warehouse_volume):
            # CHECK
            cost += (volume + fetchExtraVolume(cell, t) - warehouse_volume) * buffer_cost
            demand_volume = 0
            for item in demand[t].keys():
                demand_volume += volume_per_cu[item] * demand[t][item]
            volume -= demand_volume
            # CHECK
            volume += fetchExtraVolume(cell, t)
        else:
            break
    
    cell.cost = cost
    cell.avg_cost = cost / (cell.period_end - cell.order_date + 1) 

order_date = 0
end_reached = False
obj_val = 0
schedule = {}
while(True):
    if(order_date%7==6):
        order_date += 1
        continue
    min_cost = float('inf')
    for t in range(order_date,n):
        if(t%7==6):
            continue
        schedule[order_date] = {}
        getCost(cost_matrix[order_date][t])
        if(cost_matrix[order_date][t].avg_cost < min_cost):
            min_cost = cost_matrix[order_date][t].avg_cost
            if(t==n-1):
                schedule[order_date] = cost_matrix[order_date][t].orders
                end_reached = True
                obj_val += cost_matrix[order_date][t].cost
                break
        else:
            schedule[order_date] = cost_matrix[order_date][t-1].orders
            obj_val += cost_matrix[order_date][t-1].cost
            order_date = t
            break
    if(order_date >= n or end_reached):
        break
print(schedule)
obj_val

{0: {'01127823': 3, '01b36aba': 0, '0251c9b9': 4, '0345cdce': 10, '049bd1d6': 6, '060983da': 9, '06c3828c': 3, '07a0d713': 10, '07d20bd1': 20, '08eac3a8': 0, '09d6af7f': 1, '09dcfb6a': 3, '0b3ce2cc': 9, '0c56b941': 2, '0d300b25': 6, '0d49b412': 4, '0e4e5fc6': 0, '0f21f159': 5, '10321e91': 2, '108c8e5c': 10, '118da7e7': 2, '12db3388': 65, '142590d4': 0, '14413f5c': 5, '14ac66c0': 1, '15556a6c': 1, '15dd79a4': 0, '16254047': 11, '16e5929d': 2, '1791adf0': 2, '17a9a7fb': 1, '17b26cb3': 0, '18bdf091': 9, '18c6ae1a': 0, '19ca1986': 0, '1b0ffe97': 0, '1b588529': 0, '1c437f61': 21, '1c9715ac': 6, '1db6be05': 0, '1dfc2af7': 2, '1ea5e53b': 7, '22729ff6': 1, '230b889a': 25, '233e8f08': 3, '2379523d': 1, '28263285': 4, '282e57c5': 4, '2883c45f': 0, '2acc451d': 4, '2b317847': 19, '2c012cb4': 1, '2d8c1230': 0, '2e6ea6bf': 1, '2fb07fa8': 2, '2ff7bc91': 1, '302dbe12': 0, '303f0609': 11, '30814ac7': 4, '316bb152': 0, '3283f919': 27, '334d059e': 9, '336e39e8': 3, '34611f42': 0, '34c3b0eb': 0, '357bbf4d

677437.0999999999